In [1]:
from IPython.display import Image

## trpo vs. policy gradient 

- cons:
    - 计算量更大；
- pros:
    - 表现更稳定，收敛更快
- trust region：
    - 数值优化领域很经典的算法

### Gradient Ascent

- gradient ascent（最大化问题梯度上升，如果是最小化问题，则用梯度下降）

    $$
    \theta^\star=\arg\max_\theta J(\theta)
    $$

    - $g=\frac{\partial J(\theta)}{\partial \theta}\big|_{\theta=\theta_{old}}$，在 $\theta_{old}$ 时，计算梯度；
    - $\theta_{new}=\theta_{old}+\alpha\cdot g$，基于梯度上升，更新参数；
    - 直到梯度 $g$ 的二范数接近于0；

### Stochastic Gradient Ascent 

- stochastic gradient ascent
    - $J(\theta)=\mathbb E_S[V(S;\theta)]$ （$S$ 是随机变量）
        - 期望需要做定积分，但定积分可能没有解析解，也就是求不出期望，也就不可能算出梯度；
        - 求不出梯度，但可以求出随机梯度（stochastic gradient），随机梯度是对真实梯度的蒙特卡洛近似，
        - 用随机梯度代替梯度得到的算法就是随机梯度上升（stochastic gradient ascent）；
    - SGA
        - $s \Leftarrow$  random sampling
        - $g=\frac{\partial V(s;\theta)}{\partial \theta}\big|_{\theta=\theta_{old}}$（随机梯度）
        - $\theta_{new}=\theta_{old}+\alpha\cdot g$

### Trust Region （置信域）

$$
\theta^\star=\arg\max_\theta J(\theta)
$$

- 定义 $\mathcal N(\theta_{old})$ 是 $\theta_{old}$ 的邻域；
    - 半径为 $\delta$ 的圆；
  $$
  \mathcal N(\theta_{old})=\left\{\theta\big| \|\theta-\theta_{old}\|\leq \delta\right\}
  $$

- 如果相比较复杂的 $J(\theta)$, $L(\theta|\theta_{old})$ 是一个更为简单的函数，且 $L(\theta|\theta_{old})$ 能更好地逼近 $J(\theta)$ 在 $\mathcal N(\theta_{old})$，则 $\mathcal N(\theta_{old})$ 就可以成为置信域；
    - 用 $L(\theta|\theta_{old})$ 代替 $J(\theta)$ 可以让优化变得更容易；

- 置信域算法（Trust Regions）重复如下的两步
    - Approximation（做近似）：给定 $\theta_{old}$ 构造 $L(\theta|\theta_{old})$ 是在 $\mathcal N(\theta_{old})$ 内对 $J(\theta)$ 的近似
        - 比如用 $J(\theta)$ 的二阶泰勒展开
        - 比如用期望的蒙特卡洛近似
    - Maximization（最大化）：$\theta_{new}\leftarrow \arg\max_{\theta\in\mathcal N(\theta_{old})} L(\theta|\theta_{old})$
        - 带约束的最大化问题
            - 置信域的半径可以发生，我们可以让半径逐渐变小；
        - 每一轮都要求解这样一个最大化问题，因此置信域的算法的求解速度不快
        - 运算量会比 gd/sgd 要大，其好处在于比梯度算法更稳定

In [2]:
Image(url='https://upload.wikimedia.org/wikipedia/commons/c/cc/Mmalgorithm.jpg', width=400)

- $\theta_m$ 的邻域就是 trust region

## policy-based reinforcement learning

- $\pi_\theta(a|s)$ policy network, controlling the agent
- $V_\pi(s)=\mathbb E_{A\in \pi}[Q_\pi(s,A)]$，状态价值函数（state-value function）
    - 对谁求积分，就是消掉谁的过程，比如下面的 $A$
$$
\begin{split}
V_\pi(s)&=\mathbb E_{A\in\pi}[Q_\pi(s,A)]\\
&=\sum_a \pi_\theta(a|s) Q_\pi(s,a)
\end{split}
$$

- objective function
  - 对谁求积分，就是消掉谁的过程，比如下面的 $S$

$$
J(\theta)=\mathbb E_S[V_\pi(S)]
$$

- objective function 的推导

$$
\begin{split}
V_\pi(S)&=\sum_a\pi_\theta(a|s)Q_\pi(s,a)\\
&=\sum_a\pi_{\theta_{old}}(a|s)\frac{\pi_\theta(a|s)}{\pi_{\theta_{old}}(a|s)}Q_\pi(s,a)\\
&=\mathbb E_{A\sim\pi_{\theta_{old}}(\cdot|s)}\left[\frac{\pi_\theta(A|s)}{\pi_{\theta_{old}}(A|s)}Q_\pi(s,A)\right]
\end{split}
$$

- trpo 中最重要的公式：

$$
\begin{split}
J(\theta)&=\mathbb E_S[V_\pi(S)]\\
&=\mathbb E_S\left[\mathbb E_{A\sim\pi_{\theta_{old}}(\cdot|s)}\left[\frac{\pi_\theta(A|s)}{\pi_{\theta_{old}}(A|s)}Q_\pi(s,A)\right]\right]\\
&=\mathbb E_S\left[\mathbb E_{A}\left[\frac{\pi_\theta(A|s)}{\pi_{\theta_{old}}(A|s)}Q_\pi(s,A)\right]\right]\\
&=\mathbb E_{S,A}\left[\frac{\pi_\theta(A|s)}{\pi_{\theta_{old}}(A|s)}Q_\pi(s,A)\right]
\end{split}
$$

- $S$ 的随机性来自于状态转移（$S$ is sampled from the **state transition** of the env)
- $A$ 的随机性来自于 $\pi_{\theta_{old}}(A|S)$（$A$ is sampled from $\pi_{\theta_{old}}(A|S)$）
- 对期望 $\mathbb E_{S,A}$ 做蒙特卡洛近似；
    - trajectory（基于 $\pi_{\theta_{old}}(\cdot|s)$）：$s_1,a_1,r_1,...,s_n,a_n,r_n$
    - 蒙特卡洛近似（Step 1，approximation）
 
      $$
      L(\theta|\theta_{old})=\frac1n\sum_{i=1}^n\frac{\pi_\theta(a_i|s_i)}{\pi_{\theta_{old}}(a_i|s_i)}Q_\pi(s_i,a_i)
      $$

  

### TRPO 算法

- 巧妙地将数值优化中的 trust region 应用在策略网络 $\pi_\theta(a|s)$ 学习的优化中
- policy gradient：
    - 对参数敏感；
    - 收敛曲线上下震荡；
- more robust than policy gradient algorithms
- more sample efficient than policy gradient algorithms

- Step 1，approximation

  $$
  L(\theta|\theta_{old})=\frac1n\sum_{i=1}^n\frac{\pi_\theta(a_i|s_i)}{\pi_{\theta_{old}}(a_i|s_i)}Q_\pi(s_i,a_i)
  $$

    - 进一步对 $Q_\pi(s_i,a_i)$ 做蒙特卡洛近似
        - 一次 episode 观测到的 rewards：$r_1,r_2,\cdots,r_n$
        - 计算 discounted return（$i$ 时刻起，所有未来时刻的奖励的加权和）

          $$
          u_i=r_i+\gamma r_{i+1}+\gamma^2r_{i+2}+\cdots+\gamma^{n-i}r_n=\sum_{k=i}^n\gamma^{k-i}r_k
          $$
        - $Q_\pi(s_i,a_i)\approx u_i$

  $$
  \begin{split}
  J(\theta)&\approx L(\theta|\theta_{old})=\frac1n\sum_{i=1}^n\frac{\pi_\theta(a_i|s_i)}{\pi_{\theta_{old}}(a_i|s_i)}Q_\pi(s_i,a_i)\\
  &\approx \hat L(\theta|\theta_{old})=\frac1n\sum_{i=1}^n\frac{\pi_\theta(a_i|s_i)}{\pi_{\theta_{old}}(a_i|s_i)}u_i
  \end{split}
  $$

- Step 2, maximization

    $$
    \theta_{new}\leftarrow \arg\max_\theta \hat L(\theta|\theta_{old}), \quad s.t. \theta\in \mathcal N(\theta_{old})
    $$

    - 如何衡量 $\theta$ 与 $\theta_{old}$ 的距离
        - $\|\theta-\theta_{old}\|\leq \Delta$
        - $\frac1n\sum_{i=1}^nKL[\pi_{\theta_{old}}(\cdot|s_i)\|\pi_{\theta}(\cdot|s_i)]\leq \Delta$

In [3]:
Image(url='../imgs/trpo_summary.png', width=500)